In [50]:
import numpy as np
import pandas as pd 

In [51]:
#导入csv数据
data = pd.read_csv('data1.csv',encoding='gbk')
data.head()


,日期,收盘,开盘,高,低,交易量,涨跌幅
0,2021年4月16日,"3,426.62","3,407.19","3,432.63","3,394.13",25.86,0.81%
1,2021年4月15日,"3,398.99","3,409.61","3,409.61","3,373.09",23.4,-0.52%
2,2021年4月14日,"3,416.72","3,397.02","3,420.03","3,392.98",23.32,0.60%
3,2021年4月13日,"3,396.47","3,411.29","3,425.39","3,387.90",27.06,-0.48%
4,2021年4月12日,"3,412.95","3,445.97","3,456.16","3,403.90",30.96,-1.09%


In [52]:
#按照日期降序排序
data.sort_index(axis=0,ascending=True,inplace=True)
#选取 5列数据作为特征：收盘价 列数据作为特征：收盘价 最高价 最低价 开盘价 成交量
data=data[['收盘','高','低','开盘','交易量']]
print(data.shape)

(7411, 5)


In [53]:
print(data.head())

         收盘         高         低        开盘    交易量
0  3,426.62  3,432.63  3,394.13  3,407.19  25.86
1  3,398.99  3,409.61  3,373.09  3,409.61   23.4
2  3,416.72  3,420.03  3,392.98  3,397.02  23.32
3  3,396.47  3,425.39  3,387.90  3,411.29  27.06
4  3,412.95  3,456.16  3,403.90  3,445.97  30.96


In [54]:
#如果收盘价为0，剔除该行数据
data=data[data['收盘']!=0]
#删除nan数据
data=data.dropna()
print(data.shape)

(7411, 5)


In [55]:
#dayfeature：选取 100 天的数据
#featurenum：选取的 5个特征 *天数
#x：记录 100 天的 5个特征值
#y：记录涨或者 跌
dayfeature=100
featurenum=5*dayfeature
x=np.zeros((data.shape[0]-dayfeature,featurenum+1))
y=np.zeros((data.shape[0]-dayfeature))



In [68]:
def remove_commas(array):
    new_array = [item.replace(',', '') for item in array]
    return new_array
data=data.apply(remove_commas)
#删除交易量为'-'的行
data=data[data['交易量']!='-']


In [69]:
for i in range(0,data.shape[0]-dayfeature):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
    [[ '收盘','高','低','开盘','交易量']]).reshape((1,featurenum))
    #最后一列记录当天的开盘价
    x[i,featurenum]=data.iloc[i+dayfeature]['开盘']
for i in range(0,data.shape[0]-dayfeature):
    #如果当天收盘价高于开盘价，用1代表涨，0代表跌
    if data.iloc[i+dayfeature]['收盘']>=data.iloc[i+dayfeature]['开盘']:
        y[i]=1
    else:
        y[i]=0



In [71]:
#创建SVM，并进行10折交叉验证
from sklearn import svm
from sklearn import model_selection
clf=svm.SVC(kernel='rbf',C=1.0,gamma='auto')
result=[]
for i in range(10):
    #x和y的验证集 和测试，切分为8:2的测试集
    x_train,x_test,y_train,y_test=model_selection.train_test_split(x,y,test_size=0.2)
    #训练数据进行训练
    clf.fit(x_train,y_train)
    #将预测数据和测试集的验证数据做对比
    result.append(np.mean(y_test==clf.predict(x_test)))
print("SVM分类的正确率（每次交叉验证的结果）为:")
print(result)

SVM分类的正确率（每次交叉验证的结果）为:
[0.5994531784005468, 0.5933014354066986, 0.6124401913875598, 0.605604921394395, 0.6090225563909775, 0.5905673274094326, 0.6117566643882434, 0.6308954203691046, 0.579630895420369, 0.5830485304169515]
